# Data analysis of powder XRD data

In [10]:
import glob
import os
import re

import numpy as np
import pandas as pd
import scipy.signal as sps

from matplotlib import pyplot as plt

In [2]:
os.chdir(r'C:\Users\Pierre\Desktop\2021-03-12_powder_XRD_data')

## Reference spectra taken from the ICDD

In [4]:
ref_files = glob.glob('*.xy')

In [27]:
dref = {}
for file in ref_files:
    fm = re.match('([^\W_]+)_*', file)
    compound = fm.group(1)
    with open(file, 'r'):
        dref[compound] = pd.read_csv(file, sep='\s+', header=0, names=['2theta', 'I', 'std?'])
        
dref[compound]

,2theta,I,std?
0,1.01,0.05683,0.0
1,1.02,0.05577,0.0
2,1.03,0.05474,0.0
3,1.04,0.05374,0.0
4,1.05,0.05276,0.0
...,...,...,...
11894,119.95,0.00000,0.0
11895,119.96,0.00000,0.0
11896,119.97,0.00000,0.0
11897,119.98,0.00000,0.0


In [38]:
xlabel = r'2$\theta$ (°)'
ylabel = 'I (a.u.)'

In [39]:
%matplotlib
fig = plt.figure()
for cpd in dref.keys():
    plt.plot(dref[cpd]['2theta'], dref[cpd]['I'], label=cpd)
    plt.xlim([0,75])
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()

Using matplotlib backend: Qt5Agg


## Experimental powder XRD data

In [43]:
xp_files = glob.glob('*_flat-background.ASC')

In [146]:
compounds = {}
dxp = {}
for file in xp_files:
    fm = re.match('(\d{4}-\d{2}-\d{2})_([^\W_]+)-([^\W_]+)\w+-\w+.ASC', file)
    date = fm.group(1)
    batch = fm.group(3)
    compounds[batch] = fm.group(2)
    with open(file, 'r'):
        dxp[batch] = pd.read_csv(file, sep=' ', header=0, names=['2theta', 'I0', 'Estimated Std'])

dxp[batch]

,2theta,I0,Estimated Std
0,15.021162,353.508985,27.568158
1,15.035162,353.550882,27.577882
2,15.049162,331.818932,27.190108
3,15.063162,322.734992,27.031692
4,15.077162,334.181059,27.251657
...,...,...,...
4279,74.927162,472.999673,26.913086
4280,74.941162,373.687076,25.062410
4281,74.955162,328.636710,24.211502
4282,74.969162,289.184565,23.449254


In [84]:
batches = ['LS5695', 'LS5701', 'LS5696', 'LS5697']

In [152]:
for batch in batches:
    I = dxp[batch]['I0']
    Iref = np.mean(I)
    mult = 1.3
    Im = np.mean(I[I<Iref*mult])
    ii = 0
    while abs(Iref-Im)/Iref>0.01:
        Iref = Im
        Im = np.mean(I[I<Iref*mult])
        ii = ii+1
        if ii>100:
            break
        print(ii, Iref, Im)

    dxp[batch]['I'] = dxp[batch]['I0']-Im
    dxp[batch]

1 312.3195552760272 307.2008893883647
2 307.2008893883647 306.7936270683599
1 313.865644417357 304.34229103662386
2 304.34229103662386 303.17062259656973
1 309.18051098084044 303.62188803157835
2 303.62188803157835 303.0619490611636
1 315.1011523919672 310.24887172674687
2 310.24887172674687 309.9170183031991


## Compare measured and reported XRD patterns for pure TmVO4

In [195]:
ref = 'TmVO4'
batch = 'LS5695'
fig = plt.figure()
plt.plot(dref[ref]['2theta'], dref[ref]['I'], label='Calculated')
plt.plot(dxp[batch]['2theta'], dxp[batch]['I']*1e-2, label=batch)
plt.xlim([18,40])
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.title('XRD pattern of TmVO4-LS5695 vs pattern calculated from .cif file')
plt.legend()

In [196]:
plt.savefig('./2021-03-12_YTmVO4_analysis/2021-03-12_TmVO4_full_spectrum.pdf')

In [194]:
refs = list(dref.keys())

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True)
# fig.subplots_adjust(hspace=0.025)  # adjust space between axes

xlim = {'ax1':[18.5,19.3], 'ax2':[24.5,25.75], 'ax3':[33.25,34.25]}

for sax in ['ax1', 'ax2', 'ax3']:
    ax = eval(sax)
    xref = dref[refs[0]]['2theta']
    Iref = dref[refs[0]]['I']
    ax.plot(xref, Iref/np.max(Iref[np.logical_and(xref>xlim[sax][0], xref<xlim[sax][1])]), 
            label=f'{refs[0]} (calc.)')

    for batch in batches[:1]:
        xxp = dxp[batch]['2theta']
        Ixp = dxp[batch]['I']
        ax.plot(xxp, Ixp/np.max(Ixp[np.logical_and(xxp>xlim[sax][0], xxp<xlim[sax][1])]), 
                label=compounds[batch])
        
    ax.set_xlabel(xlabel)
    ax.set_xlim(xlim[sax])
    ax.set_ylim([0,1.1])
# ax2.set_xlim(xlim2)
# ax3.set_xlim(xlim3)
ax1.set_ylabel(ylabel)
ax2.set_title('XRD patterns of Y-substituted TmVO4 vs calculated patterns of TmVO4 and YVO4')
plt.legend()

In [197]:
plt.savefig('./2021-03-12_YTmVO4_analysis/2021-03-12_TmVO4_zoom.pdf')

## Compare measured XRD patterns of Y-substituted TmVO4 vs calculated patterns of TmVO4 and YVO4

In [155]:
refs = list(dref.keys())
fig = plt.figure()
plt.plot(dref[refs[0]]['2theta'], dref[refs[0]]['I'], label=f'{refs[0]} (calc.)')
for batch in batches[1:]:
    plt.plot(dxp[batch]['2theta'], dxp[batch]['I']*1e2/max(dxp[batch]['I']), label=compounds[batch])
plt.plot(dref[refs[1]]['2theta'], dref[refs[1]]['I'], label=f'{refs[1]} (calc.)')
plt.xlim([15,75])
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.title('XRD patterns of Y-substituted TmVO4 vs calculated patterns of TmVO4 and YVO4')
plt.legend()

In [192]:
refs = list(dref.keys())

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True)
# fig.subplots_adjust(hspace=0.025)  # adjust space between axes

xlim = {'ax1':[18.5,19.3], 'ax2':[24.5,25.75], 'ax3':[33.25,34.25]}

for sax in ['ax1', 'ax2', 'ax3']:
    ax = eval(sax)
    xref = dref[refs[0]]['2theta']
    Iref = dref[refs[0]]['I']
    ax.plot(xref, Iref/np.max(Iref[np.logical_and(xref>xlim[sax][0], xref<xlim[sax][1])]), 
            label=f'{refs[0]} (calc.)')

    for batch in batches[1:]:
        xxp = dxp[batch]['2theta']
        Ixp = dxp[batch]['I']
        ax.plot(xxp, Ixp/np.max(Ixp[np.logical_and(xxp>xlim[sax][0], xxp<xlim[sax][1])]), 
                label=compounds[batch])
        
    xref = dref[refs[1]]['2theta']
    Iref = dref[refs[1]]['I']
    ax.plot(xref, Iref/np.max(Iref[np.logical_and(xref>xlim[sax][0], xref<xlim[sax][1])]), 
            label=f'{refs[1]} (calc.)')

    ax.set_xlabel(xlabel)
    ax.set_xlim(xlim[sax])
    ax.set_ylim([0,1.1])
# ax2.set_xlim(xlim2)
# ax3.set_xlim(xlim3)
ax1.set_ylabel(ylabel)
ax2.set_title('XRD patterns of Y-substituted TmVO4 vs calculated patterns of TmVO4 and YVO4')
plt.legend()